In [1]:
# 1. 패키지 호출

import pandas as pd
import numpy as np

print(pd.__version__)
print(np.__version__)

1.0.5
1.18.5


In [2]:
#2. 데이터 업로드

from google.colab import files
uploaded = files.upload()

Saving SR_Data.csv to SR_Data.csv


In [3]:
# 3. pandas DataFrame

data = pd.read_csv('SR_Data.csv')
data.head()

,Country,Age,Year,Salary
0,Spain,27.0,3.0,48000
1,Spain,NaN,6.0,52000
2,Germany,30.0,2.0,54000
3,France,35.0,NaN,58000
4,Spain,38.0,NaN,61000


In [4]:
# 4-1. 데이터 살펴보기

data.tail()

,Country,Age,Year,Salary
5,Germany,40.0,10.0,61000
6,France,37.0,7.0,67000
7,France,44.0,15.0,72000
8,France,48.0,NaN,79000
9,Germany,50.0,21.0,83000


In [5]:
# 4-2. 데이터 정보확인
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Country  10 non-null     object 
 1   Age      9 non-null      float64
 2   Year     7 non-null      float64
 3   Salary   10 non-null     int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 448.0+ bytes


In [8]:
# 4-3. 데이터 설명보기

data.describe(include='all')

,Country,Age,Year,Salary
count,10,9.000000,7.000000,10.000000
unique,3,NaN,NaN,NaN
top,France,NaN,NaN,NaN
freq,4,NaN,NaN,NaN
mean,NaN,38.777778,9.142857,63500.000000
std,NaN,7.693793,6.817345,11597.413505
min,NaN,27.000000,2.000000,48000.000000
25%,NaN,35.000000,4.500000,55000.000000
50%,NaN,38.000000,7.000000,61000.000000
75%,NaN,44.000000,12.500000,70750.000000


In [9]:
# 5. feature/label 나누기

# feature = data[["Country", "Age", "Year"]]

# label = data["Salary"]
# label = data.iloc[0:10, 3]

feature = data.iloc[:, :-1]
label = data.iloc[:, -1]

    
print(feature)
print(label)

   Country   Age  Year
0    Spain  27.0   3.0
1    Spain   NaN   6.0
2  Germany  30.0   2.0
3   France  35.0   NaN
4    Spain  38.0   NaN
5  Germany  40.0  10.0
6   France  37.0   7.0
7   France  44.0  15.0
8   France  48.0   NaN
9  Germany  50.0  21.0
0    48000
1    52000
2    54000
3    58000
4    61000
5    61000
6    67000
7    72000
8    79000
9    83000
Name: Salary, dtype: int64


In [10]:
# 6. 비어있는 값 채우기(mean)

feature.isnull().sum()

from sklearn.impute import SimpleImputer

# 비어있는 값을 보면 평균으로 채워넣어줘야지!! 아직 평균값을 구하지 않았음
mean_imputer = SimpleImputer(strategy='mean')
 
mean_imputer.fit(feature.iloc[:, 1:]) # 데이터를 보고 평균값 계산
feature.iloc[:, 1:] = mean_imputer.transform(feature.iloc[:,1:])

feature.isnull().sum()

Country    0
Age        0
Year       0
dtype: int64

In [11]:
# 7. One hot encoding

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer([('one_hot_encoder', 
                         OneHotEncoder(),[0])], 
                       remainder='passthrough')
feature = ct.fit_transform(feature)
print(feature)

[[ 0.          0.          1.         27.          3.        ]
 [ 0.          0.          1.         38.77777778  6.        ]
 [ 0.          1.          0.         30.          2.        ]
 [ 1.          0.          0.         35.          9.14285714]
 [ 0.          0.          1.         38.          9.14285714]
 [ 0.          1.          0.         40.         10.        ]
 [ 1.          0.          0.         37.          7.        ]
 [ 1.          0.          0.         44.         15.        ]
 [ 1.          0.          0.         48.          9.14285714]
 [ 0.          1.          0.         50.         21.        ]]


In [16]:
#8. Split Data

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(feature, label,
                                                    test_size=0.2,
                                                    random_state=10)

print(X_train)
print(X_test)

[[ 0.          1.          0.         40.         10.        ]
 [ 1.          0.          0.         37.          7.        ]
 [ 1.          0.          0.         35.          9.14285714]
 [ 0.          0.          1.         38.77777778  6.        ]
 [ 0.          0.          1.         27.          3.        ]
 [ 1.          0.          0.         44.         15.        ]
 [ 0.          0.          1.         38.          9.14285714]
 [ 0.          1.          0.         50.         21.        ]]
[[ 1.          0.          0.         48.          9.14285714]
 [ 0.          1.          0.         30.          2.        ]]


In [17]:
# 9. Scaling

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train[:, 3:] = scaler.fit_transform(X_train[:, 3:])
X_test[:, 3:] = scaler.transform(X_test[:, 3:])

print(X_train[:3])

[[ 0.          1.          0.          0.20479466 -0.00678797]
 [ 1.          0.          0.         -0.27602759 -0.57697772]
 [ 1.          0.          0.         -0.59657576 -0.16969933]]


In [18]:
# 10. Train

from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train) # fit == train

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [19]:
# 11. Score

y_predicted = model.predict(X_test)

print(y_test)
print(y_predicted)

8    79000
2    54000
Name: Salary, dtype: int64
[67765.14828079 48665.74175689]


In [20]:
# 12. Evalute

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_predicted, squared=False)
mae = mean_absolute_error(y_test, y_predicted)
r2 = r2_score(y_test, y_predicted)

print(mse)
print(mae)
print(r2)

8794.208439557171
8284.554981158071
0.5050361466983773
